## Deliverable 2. Create a Customer Travel Destinations Map.

In [44]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import google_maps_key

# Configure gmaps API key
gmaps.configure(api_key=google_maps_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,72.81,81,2,14.34,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,49.66,46,20,6.91,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,79.57,75,4,8.88,light rain
3,3,Bulgan,MN,48.8125,103.5347,12.07,76,38,3.47,scattered clouds
4,4,Yulara,AU,-25.2406,130.9889,80.44,24,17,11.50,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,79.57,75,4,8.88,light rain
4,4,Yulara,AU,-25.2406,130.9889,80.44,24,17,11.50,few clouds
10,10,Tumpat,MY,6.1978,102.1710,76.98,88,20,2.30,few clouds
14,14,Saint-Philippe,RE,-21.3585,55.7679,80.19,86,98,4.23,light rain
21,21,Dingle,PH,10.9995,122.6711,77.97,84,94,6.73,overcast clouds
24,24,Avarua,CK,-21.2078,-159.7750,77.05,88,20,4.61,few clouds
30,30,Tagusao,PH,9.1924,117.8141,80.01,82,100,3.89,overcast clouds
35,35,Bambous Virieux,MU,-20.3428,57.7575,80.22,94,20,0.00,few clouds
37,37,San Ignacio,BZ,17.1561,-89.0714,89.74,62,75,6.91,broken clouds
44,44,Jacareacanga,BR,-6.2222,-57.7528,87.44,60,55,3.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                225
City                   225
Country                223
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
clean_df.count()

City_ID                223
City                   223
Country                223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"]=np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.57,light rain,-23.1203,-134.9692,NaN
4,Yulara,AU,80.44,few clouds,-25.2406,130.9889,NaN
10,Tumpat,MY,76.98,few clouds,6.1978,102.1710,NaN
14,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,NaN
21,Dingle,PH,77.97,overcast clouds,10.9995,122.6711,NaN
24,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,NaN
30,Tagusao,PH,80.01,overcast clouds,9.1924,117.8141,NaN
35,Bambous Virieux,MU,80.22,few clouds,-20.3428,57.7575,NaN
37,San Ignacio,BZ,89.74,broken clouds,17.1561,-89.0714,NaN
44,Jacareacanga,BR,87.44,broken clouds,-6.2222,-57.7528,NaN


In [46]:
hotel_df.count()

City                   223
Country                223
Max Temp               223
Current Description    223
Lat                    223
Lng                    223
Hotel Name               0
dtype: int64

In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_maps_key
}
   
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [48]:
hotel_df.count()

City                   223
Country                223
Max Temp               223
Current Description    223
Lat                    223
Lng                    223
Hotel Name             206
dtype: int64

In [49]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [50]:
clean_hotel_df.count()

City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             206
dtype: int64

In [51]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# # 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# # 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [53]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))